#Make a smaller development set

In [ ]:
import json
import random
import os
from collections import Counter
from typing import Tuple, List, Any

In [ ]:
%cd /content/drive/MyDrive/Neural-CF

user_path = "./Yelp-Dataset/yelp_academic_dataset_user.json"
business_path = "./Yelp-Dataset/yelp_academic_dataset_business.json"
review_path = "./Yelp-Dataset/yelp_academic_dataset_review.json"

/content/drive/MyDrive/Neural-CF


In [ ]:
with open(user_path, 'r', encoding="utf-8") as f:
    user_data = list(map(json.loads, f.readlines()))

In [ ]:
with open(business_path, 'r', encoding="utf-8") as f:
    business_data = list(map(json.loads, f.readlines()))

In [ ]:
with open(review_path, 'r', encoding="utf-8") as f:
    review_data = list(map(json.loads, f.readlines()))

In [ ]:
# # we prioritize restaurants with the most reviews
# business_cnt = Counter([review["business_id"] for review in review_data])
# business_cnt = list(business_cnt.items())
# business_cnt.sort(key=lambda x: x[1], reverse=True)

# we prioritize users with most reviews...
user_cnt = Counter(review["user_id"] for review in review_data)
user_cnt = list(user_cnt.items())
user_cnt.sort(key=lambda x: x[1], reverse=True)

In [ ]:
subset_user_id = {user_id for user_id, _ in user_cnt[:1000]}
subset_user_data = [user for user in user_data if user["user_id"] in subset_user_id]
subset_review_data = [review for review in review_data if review["user_id"] in subset_user_id]

subset_business_id_cnt = Counter([review["business_id"] for review in subset_review_data])
subset_business_id_cnt = list(subset_business_id_cnt.items())
subset_business_id_cnt.sort(key=lambda x: x[1], reverse=True)

subset_business_id = {business_id for business_id, _ in subset_business_id_cnt[:1000]}
subset_business_data = [business for business in business_data if business["business_id"] in subset_business_id]

subset_review_data = [review for review in review_data if review["user_id"] in subset_user_id and review["business_id"] in subset_business_id]
print(len(subset_business_data))
print(len(subset_review_data))
print(len(subset_user_data))

1000
48671
1000


In [ ]:
output_folder = "./Yelp-Dataset/subset/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def save_data(
    data: Tuple[List[Any], ...],
    data_folder: str,
    filenames=("yelp_academic_dataset_user.json", "yelp_academic_dataset_business.json", "yelp_academic_dataset_review.json"),
):
    for d, f in zip(data, filenames):
        open(os.path.join(data_folder, f), "w").write("\n".join(map(json.dumps, d)))

save_data((subset_user_data, subset_business_data, subset_review_data), output_folder, filenames=("subset_user.json", "subset_business.json", "subset_review.json"))
